In [1]:
%load_ext autoreload
%autoreload 2

# mount drive
from google.colab import drive
drive.mount('/content/drive')

# cd into project directory
%cd /content/drive/My\ Drive/Georgia_Tech/Spring_2021/cs_8903_nlp

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1-q8XFdyP4Idrk0nziCyjQ4pTlg7U3vXF/cs_8903_nlp


In [2]:
!pip install transformers
!pip install datasets

import torch
import pandas as pd
import numpy as np

     |████████████████████████████████| 1.8MB 7.6MB/s 
     |████████████████████████████████| 890kB 37.9MB/s 
     |████████████████████████████████| 3.2MB 51.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=004b217ba17de25959f201605577d3feab739fd1cb652e18465243f1e1a9f403
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 184kB 8.3MB/s 
     |████████████████████████████████| 102kB 8.7MB/s 
     |████████████████████████████████| 20.7MB 1.2MB/s 
     |████████████████████████████████| 245kB 58.9MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
True


In [3]:
from data_cleaning import *

# Read and Clean Data
trndf = pd.read_csv("SBIC.v2.trn.csv")
devdf = pd.read_csv("SBIC.v2.dev.csv")

clean_target_minority(trndf)
clean_target_stereotype(trndf)
clean_post(trndf)
create_text_column(trndf)

clean_target_minority(devdf)
clean_target_stereotype(devdf)
clean_post(devdf)
create_text_column(devdf)

trndf[['text']].sample(frac=1).to_csv('sbic_train_text.csv', index=False)
devdf[['text']].sample(frac=1).to_csv('sbic_dev_text.csv', index=False)

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [4]:
from datasets import load_dataset

datasets = load_dataset("csv", data_files={"train": "sbic_train_text.csv", "validation": "sbic_dev_text.csv"})
print(datasets)

Using custom data configuration default-82f84e65b7e956dc


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-82f84e65b7e956dc/0.0.0/965b6429be0fc05f975b608ce64e1fa941cc8fb4f30629b523d2390f3c0e1a93. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 112900
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 16738
    })
})


In [5]:
# Tokenize the text
from transformers import AutoTokenizer

# Store the model we want to use
MODEL_NAME = "openai-gpt"

# We need to create the model and tokenizer
# model = OpenAIGPTLMHeadModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
categorical_tokens = [LEWDY,LEWDN,OFFY,OFFN,INTY,INTN,GRPY,GRPN,INGY,INGN]
special_tokens = {'sep_token': SEP, \
                  'bos_token': BOS, \
                  'eos_token': EOS, \
                  'pad_token': PAD, \
                  'additional_special_tokens': categorical_tokens}
tokenizer.add_special_tokens(special_tokens)

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
print(tokenized_datasets)





DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 112900
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 16738
    })
})


In [6]:
# Normalize the text length as Blocks
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    if total_length % block_size:
      remainder = block_size - (total_length % block_size)
    else:
      remainder = 0
    
    pad_extension = [tokenizer.pad_token_id for _ in range(remainder)]
    attention_extension = [0 for _ in range(remainder)]

    concatenated_examples['input_ids'].extend(pad_extension)
    concatenated_examples['attention_mask'].extend(attention_extension)
    total_length += remainder
    
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

print(lm_datasets)





DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 34858
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 5213
    })
})


In [ ]:
# Load PreTrained Model and train it
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(
    "model/sbic_gpt_baseline",
    evaluation_strategy = 'steps',
    eval_steps = 1000,
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

trainer.train()

Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Runtime,Samples Per Second
1000,3.423500,3.109340,57.157000,91.205000
2000,3.003700,2.861863,57.088700,91.314000
3000,2.875800,2.800547,57.088000,91.315000
4000,2.796000,2.774509,57.225400,91.096000
